In [228]:
import pandas as pd

In [229]:
games = pd.read_csv('games_stats.csv', index_col=0)  # read data starting at 1st index

In [230]:
# should have 1520 rows (games): 38 (games per season) * 20 (teams) * 2 (seasons) = 1520
# 6 teams relegated over 2 seasons (so 36*2=72 games for all teams but 6 relegated=38 games)
games.shape

(1520, 27)

In [231]:
# delete useless data
del games['comp']
del games['notes']

In [232]:
games.dtypes

date             object
time             object
round            object
day              object
venue            object
result           object
gf                int64
ga                int64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
season            int64
team             object
dtype: object

In [233]:
# Predictors: (Convert date object properties to machine readable, computable data (INTS))
games['date'] = pd.to_datetime(games['date'])  # override date to datetime
games['venue_c'] = games['venue'].astype('category').cat.codes  # venue column: translates venue (str) -> (category) -> (int): 0 or 1
games['opp_c'] = games['opponent'].astype('category').cat.codes  # opponenet column: translates to code (each team has own code)
games['hour_c'] = games['time'].str.replace(':.+', '', regex=True).astype('int')  # hour column: remove minutes from time (just hour)
games['day_c'] = games['date'].dt.dayofweek  # day column: gets day of week as int
games['target'] = (games['result'] == 'W').astype('int')  # find if team won (1), lost (0), or tied (0): bool -> int

In [234]:
games

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,fk,pk,pkatt,season,team,venue_c,opp_c,hour_c,day_c,target
0,2022-08-13,21:00,Matchweek 1,Sat,Home,D,0,0,Rayo Vallecano,1.9,...,1.0,0,0,2023,Barcelona,1,16,21,5,0
1,2022-08-21,22:00,Matchweek 2,Sun,Away,W,4,1,Real Sociedad,2.1,...,0.0,0,0,2023,Barcelona,0,18,22,6,1
2,2022-08-28,19:30,Matchweek 3,Sun,Home,W,4,0,Valladolid,2.6,...,1.0,0,0,2023,Barcelona,1,21,19,6,1
3,2022-09-03,21:00,Matchweek 4,Sat,Away,W,3,0,Sevilla,4.0,...,2.0,0,0,2023,Barcelona,0,19,21,5,1
5,2022-09-10,18:30,Matchweek 5,Sat,Away,W,4,0,Cádiz,3.4,...,0.0,0,0,2023,Barcelona,0,7,18,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2022-04-30,14:00,Matchweek 34,Sat,Home,W,2,1,Villarreal,2.4,...,0.0,0,0,2022,Alaves,1,22,14,5,1
36,2022-05-07,18:30,Matchweek 35,Sat,Away,L,0,4,Celta Vigo,0.5,...,0.0,0,0,2022,Alaves,0,6,18,5,0
37,2022-05-11,19:00,Matchweek 36,Wed,Home,W,2,1,Espanyol,2.6,...,2.0,0,0,2022,Alaves,1,9,19,2,1
38,2022-05-15,19:30,Matchweek 37,Sun,Away,L,1,3,Levante,0.5,...,1.0,0,0,2022,Alaves,0,13,19,6,0


In [235]:
# INITIAL MODEL:

In [236]:
from sklearn.ensemble import RandomForestClassifier  # model to pick up non-linearities in the data
rfc = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)  # 50 decision trees, 10 sample split, same result on mult run

In [237]:
# split training and test data (all test set data reads after training set: past -> future)
train_set = games[games['date'] < '2023-01-01']  # trainig set: all matches before 2023 (2022 season)
test_set = games[games['date'] > '2023-01-01']  # test set: all matches after 2023 (2023 season)

In [238]:
predictors = ['venue_c', 'opp_c', 'hour_c', 'day_c']  # created predictors
rfc.fit(train_set[predictors], train_set['target'])  # train model with predictors based on wins/losses

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [239]:
preds = rfc.predict(test_set[predictors])  # generate predictions based on test data

In [240]:
# Determine Accuracy:
from sklearn.metrics import accuracy_score  # metric that determines accuracy of predictions vs. data
accuracy = accuracy_score(test_set['target'], preds) # accuracy based on W/L
accuracy

0.6086956521739131

In [241]:
# Data frame to understand accuracy:
combined_data = pd.DataFrame(dict(actual=test_set['target'], prediction=gen_preds))  # data frame that combines actual and predicted values

In [242]:
pd.crosstab(index=combined_data['actual'], columns=combined_data['prediction'])  # 2-way table that shows prediction determination process

prediction,0,1
actual,,
0,226,55
1,125,54


In [243]:
# REVISED MODEL

In [244]:
# Revise accuracy model (based on testing for wins):
from sklearn.metrics import precision_score  # perc of time team won when win predicted
precision_score(test_set['target'], preds)  # model to be improved

0.4954128440366973

In [245]:
# Split by team:
team_games = games.groupby('team')  # create a data frame for each team
team = team_games.get_group('Barcelona').sort_values('date')  # test team

In [246]:
# Compute rolling averages for each team (for better predictions) & assign to new column:
def rolling_avrgs(team, cols, n_cols):
    team = team.sort_values('date')  # sort in ascending order
    roll_stats = team[cols].rolling(3, closed='left').mean()  # compute rolling avrg for previous 3 weeks (ignore current week)
    team[n_cols] = roll_stats  # assign back to original df w/ new names
    team = team.dropna(subset=n_cols)  # drop any missing data/values (edge case) removing rows w/ missing
    return team

In [247]:
cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']  # columns to compute rolling averages for (shots & goals)
n_cols = [f'{c}_roll' for c in cols]  # adds rolling to end of original col name (for rolling averages)

In [248]:
n_cols

['gf_roll',
 'ga_roll',
 'sh_roll',
 'sot_roll',
 'dist_roll',
 'fk_roll',
 'pk_roll',
 'pkatt_roll']

In [249]:
# test rolling average application:
rolling_avrgs(team, cols, n_cols).head()

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,day_c,target,gf_roll,ga_roll,sh_roll,sot_roll,dist_roll,fk_roll,pk_roll,pkatt_roll
4,2021-09-20,21:00,Matchweek 5,Mon,Home,D,1,1,Granada,1.8,...,0,0,2.333333,1.333333,9.666667,4.666667,14.900000,0.333333,0.000000,0.000000
5,2021-09-23,22:00,Matchweek 6,Thu,Away,D,0,0,Cádiz,0.4,...,3,0,1.333333,1.000000,11.000000,4.000000,15.600000,0.666667,0.000000,0.000000
6,2021-09-26,16:15,Matchweek 7,Sun,Home,W,3,0,Levante,2.9,...,6,1,1.000000,0.666667,10.000000,3.666667,16.066667,0.666667,0.000000,0.000000
8,2021-10-02,21:00,Matchweek 8,Sat,Away,L,0,2,Atlético Madrid,0.7,...,5,0,1.333333,0.333333,14.000000,6.333333,16.633333,0.666667,0.333333,0.333333
9,2021-10-17,21:00,Matchweek 9,Sun,Home,W,3,1,Valencia,1.9,...,6,1,1.000000,0.666667,11.333333,5.000000,18.733333,1.000000,0.333333,0.333333


In [250]:
# apply to all matches
games_rolling = games.groupby('team').apply(lambda x : rolling_avrgs(x, cols, n_cols))  # compute rolling averages for each match and team
games_rolling = games_rolling.droplevel('team')  # drop extra index level
games_rolling.index = range(games_rolling.shape[0])  # assigns values from 0 - rows to be new indeces (unique values for each index)
games_rolling

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,day_c,target,gf_roll,ga_roll,sh_roll,sot_roll,dist_roll,fk_roll,pk_roll,pkatt_roll
0,2021-09-18,21:00,Matchweek 5,Sat,Home,L,0,2,Osasuna,0.5,...,5,0,0.333333,2.666667,9.333333,3.000000,17.633333,0.666667,0.333333,0.333333
1,2021-09-22,19:30,Matchweek 6,Wed,Away,L,0,1,Espanyol,0.5,...,2,0,0.000000,2.000000,8.666667,3.000000,19.033333,0.333333,0.000000,0.000000
2,2021-09-25,14:00,Matchweek 7,Sat,Home,W,1,0,Atlético Madrid,0.7,...,5,1,0.000000,2.000000,8.000000,2.333333,19.066667,0.333333,0.000000,0.000000
3,2021-10-01,21:00,Matchweek 8,Fri,Away,L,0,1,Athletic Club,0.9,...,4,0,0.333333,1.000000,7.333333,2.000000,17.800000,0.000000,0.000000,0.000000
4,2021-10-18,19:00,Matchweek 9,Mon,Home,L,0,1,Betis,0.9,...,0,0,0.333333,0.666667,10.333333,2.666667,15.066667,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,2023-05-13,18:30,Matchweek 34,Sat,Home,W,5,1,Athletic Club,3.2,...,5,1,2.666667,1.333333,16.333333,6.333333,17.433333,1.000000,0.000000,0.666667
1444,2023-05-20,14:00,Matchweek 35,Sat,Away,W,2,1,Girona,1.9,...,5,1,3.000000,1.000000,12.000000,6.000000,17.400000,1.000000,0.000000,0.333333
1445,2023-05-24,19:30,Matchweek 36,Wed,Home,W,2,0,Cádiz,2.4,...,2,1,2.666667,1.000000,12.666667,6.666667,18.066667,1.333333,0.000000,0.000000
1446,2023-05-28,19:00,Matchweek 37,Sun,Away,L,1,2,Rayo Vallecano,0.4,...,6,0,3.000000,0.666667,13.333333,7.666667,18.600000,1.666667,0.000000,0.000000


In [251]:
# Make easier to iterate on algorithm (don't repeat train / test code, function for starting operations)
def make_pred(data, predictors):
    train = data[data["date"] < '2023-01-01']  # season 1 (training set)
    test = data[data["date"] > '2023-01-01']  # season 2 (test set)
    rfc.fit(train[predictors], train["target"])   # fit model
    preds = rfc.predict(test[predictors])  # create predictions
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)  # combine predictions and actuals
    error = precision_score(test["target"], preds)  # calculate precision
    return combined, error

In [252]:
# Test improvements:
combined, error = make_pred(games_rolling, predictors + n_cols)
error

0.41975308641975306

In [253]:
# IMPROVE MODEL BY BOTH SIDES OF MATCH:

In [254]:
# merge columns from games_rolling to combined for data analyzation
combined = combined.merge(games_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [257]:
combined.head()

,actual,predicted,date,team,opponent,result
47,0,0,2023-01-08,Almeria,Real Sociedad,L
48,0,0,2023-01-15,Almeria,Atlético Madrid,D
49,0,0,2023-01-23,Almeria,Valencia,D
50,1,1,2023-01-27,Almeria,Espanyol,W
51,0,0,2023-02-06,Almeria,Rayo Vallecano,L


In [259]:
# Reassign team names in-case of mis-name within data
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Barcelona FC": "Barcelona", 
              "Real Madrid FC": "Real Madrid", 
              "Sevilla FC": "Sevilla", 
              "Celta Vigo FC": "Celta Vigo", 
              } 
mapping = MissingDict(**map_values)

In [268]:
combined["new_team"] = combined["team"].map(mapping)  # create new (corrected) team_x and team_y columns
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])  # add columns for predictions for each team in game

In [262]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,0,0,2023-01-09,Athletic Club,Osasuna,D,Athletic Club,0,0,Osasuna,Athletic Club,D,Osasuna
1,0,0,2023-01-14,Athletic Club,Real Sociedad,L,Athletic Club,1,0,Real Sociedad,Athletic Club,W,Real Sociedad
2,0,0,2023-01-22,Athletic Club,Real Madrid,L,Athletic Club,1,0,Real Madrid,Athletic Club,W,Real Madrid
3,0,0,2023-01-29,Athletic Club,Celta Vigo,L,Athletic Club,1,0,Celta Vigo,Athletic Club,W,Celta Vigo
4,1,0,2023-02-03,Athletic Club,Cádiz,W,Athletic Club,0,0,Cadiz,Athletic Club,L,Cadiz
...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,1,0,2023-05-13,Villarreal,Athletic Club,W,Villarreal,0,0,Athletic Club,Villarreal,L,Athletic Club
364,1,0,2023-05-20,Villarreal,Girona,W,Villarreal,0,0,Girona,Villarreal,L,Girona
365,1,0,2023-05-24,Villarreal,Cádiz,W,Villarreal,0,0,Cadiz,Villarreal,L,Cadiz
366,0,0,2023-05-28,Villarreal,Rayo Vallecano,L,Villarreal,1,0,Rayo Vallecano,Villarreal,W,Rayo Vallecano


In [270]:
# FINAL MODEL (compare predictions twice for one match, model more confident on predicting outcomes for win and loss):
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()  # merge data frame with itself

actual_x
0    31
1    22
Name: count, dtype: int64

In [272]:
31 / 53  # 58% accurate 

0.5849056603773585